In [10]:
import os
import pandas as pd
import json
import datetime

In [2]:
base_path = "/home/seb/thesis/mitmdump_sample"

### Parse the edge data

In [3]:
data = {"source": [], "target": [], "timestamp": []}
for user_id in os.listdir(base_path):
    if os.path.exists(f"{base_path}/{user_id}/followings"):
        for other_user_id in os.listdir(f"{base_path}/{user_id}/followings"):
            with open(f"{base_path}/{user_id}/followings/{other_user_id}", "r") as f:
                data["source"].append(user_id)
                data["target"].append(other_user_id)
                data["timestamp"].append(json.load(f)["createdOn"])
    if os.path.exists(f"{base_path}/{user_id}/followers"):
        for other_user_id in os.listdir(f"{base_path}/{user_id}/followers"):
            with open(f"{base_path}/{user_id}/followers/{other_user_id}", "r") as f:
                data["source"].append(other_user_id)
                data["target"].append(user_id)
                data["timestamp"].append(json.load(f)["createdOn"])

In [6]:
df = pd.DataFrame(data)
df = df.drop_duplicates()
df.to_csv("network.csv", index=False)

### Parse node attributes

In [11]:
d = {uid: {} for uid in pd.concat([df["source"], df["target"]]).unique()}
for uid in pd.concat([df["source"], df["target"]]).unique():
    if os.path.exists(f"mitmdump_sample/{uid}/bio.json"):
        with open(f"mitmdump_sample/{uid}/bio.json", "r") as f:
            bio = json.load(f)
            if "gender" in bio["extensionDTO"].keys():
                d[uid]["gender"] = "".join(i.lower() for i in bio["extensionDTO"]["gender"] if i.isalpha())
            else:
                d[uid]["gender"] = ""
            d[uid]["age"] = int(bio["age"])
            d[uid]["post_count"] = int(bio["createdQuestionCount"])
            d[uid]["time_on_platform"] = (datetime.date(2024, 3, 1) - datetime.datetime.strptime(bio["createdOn"].split("T")[0], "%Y-%m-%d").date()).days
            #if "latestMoodPackageId" in bio["extensionDTO"].keys():
            #    d[uid]["latest_mood"] = int(bio["extensionDTO"]["latestMoodPackageId"])
            #else:
            #    d[uid]["latest_mood"] = None

In [12]:
#map genders

female = ["female", "fem", "sheher", "f", "queen", "gal", "spicygal", "mama", "witch", "lady", "chick", "woman", "women", "girl", "seductress", "whore", "slut", "stargirl", "goddess",
          "tumblrgirl", "princess", "adultfemale", "bimbosuccubus", "vampireprincess", "milf", "cakegirl", "femme", "damsel", "highlady", "puppygirl", "catgirl", "girlfailure",
          "clownfemale", "princesse", "girlie", "cumdumpster", "sillygirl", "femaleharuspex", "iamgorl", "coolgirl", "nymph", "needygirl", "grl", "mommy", "guttergirl", "grandma",
          "cumdump", "gothslut", "girlfreak", "slutbeast", "bitch", "demonqueen", "weirdgirl", "girlboss", "girlthing", "womanmoment", "fembot", "lovergrill", "gothgirl",
          "thatbitch", "girly", "vampprincess", "sadgirl", "fairygirl", "gf", "kittygirl", "justagirl", "girlgamer", "mom", "sceneslut", "dumbcunt", "dumbcumslut", "dumbbitch",
          "ambientgirl", "kafkaesquegirl", "unstablebitch", "hardfemme", "devilgirl", "vampiress", "girlishthing", "futurecatlady", "bogwitch"]
male = ["male", "man", "men", "m", "hehim", "daddy", "boyfreak", "manthing", "fuckboy", "straightmale", "cooldude", "catboy", "boything", "4th_run_logsvampiredad",
        "ratboy", "boi", "boygenius", "foxboy", "dude", "justalilguy", "dollboy", "maleabomination", "guy", "dawg", "neoboy", "dogboyhimbo", "coolguy", "cowboy", "sillyguy",
        "boy", "boytoy", "dogboy", "carguy", "sillyman", "guybro", "starboy", "boyloser", "puppyboy", "freakguy", "boiii", "sillydude", "catguy", "businessman",
        "sparkleboy", "malehehim", "catboi", "masculine", "hehimpussy", "lilguy", "manchild", "prince", "guything", "sillyguytm", "computerboy", "cuntboy", "malecow", "wolfman",
        "coyoteboy", "aguyhethey", "hesaboy", "cismale", "somenerdyguy", "manwhore", "manbutwrong", "catdogboytoy"]
nonbinary = ["nb", "shethey", "nan", "nonbinary", "agender", "trans", "transmasc", "transfem", "transman", "genderfluid", "hethem", "xenogenders", "genderqueer",
             "hethey", "gay", "lesbian", "apagender", "theythem", "demigirl", "transgirlthing", "fiestyfag", "femboymale", "genderfae", "nonbinaryman", "nonbinarywoman",
             "transnb", "genderneutral", "anypronouns", "pangender", "bisexual", "nbwoman", "nbman", "androgyne", "demiman", "transmascenby", "agenderfluid", "transguy",
             "bigender", "transmasculine", "pansexual", "nønbinary", "theyithe", "demiboy", "mtftrans", "it", "heit", "ftm", "androgynous", "genderflux", "bimale", "theyit",
             "noncomforming", "enbytranswomen", "transagender", "sheherit", "theyshe", "sheherhim", "nonbinaryfairy", "dyke", "faggot", "transmtf", "queer", "girlfag",
             "transgender", "nbmasc", "transfemale", "confused", "enbytransmasc", "transmascbutch", "fluid", "nbmale", "girlflux", "transfemme", "neagender", "other",
             "nbmascflux", "pretransmtf", "transgenderer", "transneumasc", "nbdemonboy", "agenderdemiguy", "notsure", "eldritchlesbian", "queerasfuck", "notheythem",
             "transmascmale", "shetheyhe", "maleish", "girlboything", "techgender", "nonbianary", "ittwink", "hetheylgbt", "afabnonbinary", "boyflux", "fag", "intersexenby",
             "femalebisexual", "transmascthing", "nonbinarything", "generfluid", "genderlessmasc", "intersex", "dykeboy", "none", "unlabeled", "girlboy", "gendervoid",
             "hehimgirlypop", "bigenders", "femboy", "faedemigirl", "catgirlfoxboy", "boygirl", "homo", "usuallyagirl", "bi", "boygirlthing", "they", "xeno", "malewife",
             "demigirlace", "sapphic", "nonbinarycatboy", "transdude", "mansideofenby", "girlish", "girlboyangel", "bimboyboyfail", "girboylnt", "hetheyit", "transmascnb",
             "gendrfluidfemme", "gfluid", "itthey", "nonbinaryguy", "nonbinaryfem", "boyishdisgrace", "womanman", "guyish", "lesbo", "heshethey", "femmeenby", "vampirefaggot",
             "transmale", "nobinary", "demifluid", "demigender", "femmenonbinary", "nbyboyflux", "tranny", "omnigender", "fagdyke", "masclesbian", "genderless", "amenace",
             "genderfree", "nogender", "gatygender", "xhexer", "nbtrans", "transnonbinary", "nbtransman", "andro", "dual", "multigender", "agenderafab", "maleftm", "demimale",
             "neutral", "transdemiboy", "genderfaun", "nby", "xenofemale", "libramasculine", "eviasgender", "enbytransman", "nbfem", "theyhe", "transnbagender", "demigenderf",
             "enbytheythem", "unlabelled", "neutrandrogyne", "demifemme", "heshe", "girlygirlboy", "theysheᵕ", "crossdressmtf", "idk", "nbtheythem", "anygender", "traggot",
             "transftm", "agendercatgirl", "lilbinary", "idunno", "transmascsextoy", "anyprnns", "transneutral", "nbtheyhe", "nobiney", "nbtransmasc", "unknown", "mascnonbinary",
             "sissy", "boywife", "sissy", "nonbinaryqueer", "intersexman", "agenderboygirl", "nonbinarypony", "fagboy", "gaeboy", "ithe", "transgirl", "clownfag", "gaytheythem",
             "demiagender", "transwoman", "vampireheshe", "nbgenderfluid", "tomboy", "ititsany", "voidmasc", "acepilot", "theyheit", "enbygirl", "feminineboy", "shehim",
             "nbitransmasc", "transfemboy", "shetheybug", "ftmnb", "afabbigender", "demigurl", "queermo", "nombinary", "therhythmthief", "transmascdummy", "transfemnby",
             "biswitch", "transmaschehim", "demiguy", "mascagender", "lesbean", "bigolnb", "kittennonbinary"]
gender_map = dict()
for g in [x for x in d.values()]:
    try:
        g = g["gender"]
        
        if g in female:
            gender_map[g] = "female"
        elif g in male:
            gender_map[g] = "male"
        elif g in nonbinary:
            gender_map[g] = "nonbinary"
        elif g == "":
            gender_map[g] = "empty"
        else:
            gender_map[g] = "other"
    except:
        continue

In [13]:
#convert attribute dict to dataframe
attribs = pd.DataFrame(d).T.reset_index(names="user_id")
#make sure there are no missing values
attribs = attribs[~attribs["post_count"].isna()]
#map genders
attribs["gender"] = attribs["gender"].map(gender_map)
#convert dtypes
attribs.loc[:, attribs.columns!="gender"] = attribs.drop(["gender"], axis=1).astype(int)
attribs = attribs.reset_index(drop=True)
#write to file
attribs.to_csv("node_attributes.csv", index=False)

/tmp/ipykernel_2186448/2326688876.py:8: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  attribs.loc[:, attribs.columns!="gender"] = attribs.drop(["gender"], axis=1).astype(int)
